2. The city of Los Angeles recently enacted a new hotel worker protection ordinance, impacting workload limits for hotel attendants in the region. Based on this ordinance, the Beverly Hills Four Seasons is updating its scheduling process to avoid incurring penalties. The new regulations stipulate that:
- All attendants receive a full 8 hours pay even if they work less then this. They are also allowed to work a maximum of 2 hours of overtime (in hourly increments) at 1.5 times their hourly wage.
- An attendant may clean between 2 and 4 floors per day. Attendants who clean more than two rooms (i.e., three of four) receive an additional $75 per floor.
- Each attendant can clean 3500 square feet of rooms per day. If they exceed this value, they double their regular hourly wage (this does not include overtime hours).

Data on rooms, including square footage, floor location, and the cleaning time (hours), has been provided in the file hotels.csv. Suppose there are 8 attendants working today, each earning an hourly wage of $25. Formulate and solve a binary program to minimize the hotel’s staffing cost while ensuring all rooms are cleaned and the new ordinance requirements are accounted for.


In [1]:
import pandas as pd
hotels = pd.read_csv('https://raw.githubusercontent.com/mn42899/operations_research/refs/heads/main/hotels.csv')
hotels.head()

,Room_ID,Floor,Square_Feet,Cleaning_Time_Hours
0,1,3,682,1.814545
1,2,9,223,0.562727
2,3,7,506,1.334545
3,4,1,561,1.484545
4,5,14,424,1.110909


In [2]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Room_ID              52 non-null     int64  
 1   Floor                52 non-null     int64  
 2   Square_Feet          52 non-null     int64  
 3   Cleaning_Time_Hours  52 non-null     float64
dtypes: float64(1), int64(3)
memory usage: 1.8 KB


In [3]:
hotels.describe()

,Room_ID,Floor,Square_Feet,Cleaning_Time_Hours
count,52.000000,52.000000,52.000000,52.000000
mean,26.500000,8.307692,479.326923,1.261801
std,15.154757,4.007534,157.321916,0.429060
min,1.000000,1.000000,219.000000,0.551818
25%,13.750000,5.000000,338.500000,0.877727
50%,26.500000,9.000000,486.000000,1.280000
75%,39.250000,12.000000,620.750000,1.647500
max,52.000000,14.000000,740.000000,1.972727


a) Can you explain how the costs associated with the first two regulations can be correctly tracked using only binary variables? That is, describe how the overtime constraints should work. 


b) You will also need a binary variable, fik to capture whether attendant i has been assigned to floor k. This helps to determine how many floors attendant i has been assigned to. Write down the constraint such that if attendant i is assigned to at least one room on floor k, then fik = 1.

In [4]:
from gurobipy import Model, GRB, quicksum

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
num_floors = hotels["Floor"].nunique()  # Determine the number of unique floors

big_M = 10  # Large constant for the Big-M constraint

# Create Gurobi model
model = Model("Floor Assignment Tracking")

# Decision Variables

# Binary variable: X[j, i] = 1 if room j is assigned to attendant i
X = model.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")

# Binary variable: F_ik = 1 if attendant i has been assigned at least one room on floor k
F_ik = model.addVars(num_attendants, num_floors, vtype=GRB.BINARY, name="FloorAssigned")

# Constraint: If any room on floor k is assigned to attendant i, then F_ik[i, k] = 1
for i in range(num_attendants):
    for k in range(num_floors):
        model.addConstr(
            quicksum(X[j, i] for j in range(num_rooms) if hotels.loc[j, "Floor"] == k) <= F_ik[i, k] * big_M,
            name=f"FloorAssignment_{i}_{k}"
        )

# Solve the model (if needed for testing)
model.optimize()

# Output Results
if model.status == GRB.OPTIMAL:
    for i in range(num_attendants):
        for k in range(num_floors):
            print(f"Attendant {i} assigned to Floor {k}: {F_ik[i, k].x}")
else:
    print("No optimal solution found.")

Set parameter Username
Set parameter LicenseID to value 2610046
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 112 rows, 528 columns and 504 nonzeros
Model fingerprint: 0xdc4675fd
Variable types: 0 continuous, 528 integer (528 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Attendant 0 assigned to Floor 0: -0.0
Attendant 0 assigned to Floor

c) What type of constraint is the third regulation and why?

d) Considering the cost of violating the regulations, in what order do you think penalties would be incurred if they become necessary? How would this ordering change if attendants received double their regular hourly wage (instead of time and a half) for overtime hours worked?


In [9]:
from gurobipy import Model, GRB, quicksum

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
num_floors = hotels["Floor"].nunique()  # Number of unique floors

hourly_wage = 25
max_overtime = 2  # Maximum allowed overtime hours
max_sq_ft = 3500  # Maximum square footage before pay doubles
floor_bonus = 75  # Additional payment per floor beyond 2
big_M = 10  # Large constant for Big-M constraints

# Function to create and solve the model for a given overtime rate
def solve_scenario(overtime_rate_multiplier):
    model = Model(f"Hotel Cleaning Optimization - OT {overtime_rate_multiplier}x")

    # Decision Variables
    X = model.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")
    O = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
    S = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")
    
    # Binary variable to track if attendant i has been assigned to floor k
    F_ik = model.addVars(num_attendants, num_floors, vtype=GRB.BINARY, name="FloorAssigned")

    # Binary variables to track floor bonus and square footage penalty
    FloorBonusFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="FloorBonusFlag")
    SqFtExceedFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="SqFtExceedFlag")

    # Overtime pay rate
    overtime_rate = overtime_rate_multiplier * hourly_wage

    # Constraints

    # Each room must be assigned to exactly one attendant
    for i in range(num_rooms):
        model.addConstr(quicksum(X[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

    # Track total cleaning time per attendant and enforce overtime constraints
    for j in range(num_attendants):
        total_time = quicksum(X[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
        model.addConstr(total_time - O[j] <= 8, name=f"WorkHours_{j}")
        model.addConstr(O[j] <= max_overtime, name=f"MaxOvertime_{j}")

        # Ensure attendants do not exceed the square footage limit before pay doubles
        total_sq_ft = quicksum(X[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
        model.addConstr(S[j] == total_sq_ft, name=f"TotalSqFt_{j}")
        model.addConstr(S[j] <= max_sq_ft + SqFtExceedFlag[j] * 10000, name=f"MaxSqFt_{j}")

        # Ensure attendants clean between 2 and 4 floors per day
        model.addConstr(quicksum(F_ik[j, k] for k in range(num_floors)) >= 2, name=f"MinFloors_{j}")
        model.addConstr(quicksum(F_ik[j, k] for k in range(num_floors)) <= 4, name=f"MaxFloors_{j}")

        # Set FloorBonusFlag to 1 if more than 2 floors are cleaned
        model.addConstr(quicksum(F_ik[j, k] for k in range(num_floors)) - 2 <= FloorBonusFlag[j] * 100, name=f"FloorBonusConstraint_{j}")

    # **Enforce floor assignment tracking**
    for j in range(num_attendants):
        for k in range(num_floors):
            model.addConstr(
                quicksum(X[i, j] for i in range(num_rooms) if hotels.loc[i, "Floor"] == k) <= big_M * F_ik[j, k],
                name=f"FloorAssignment_{j}_{k}"
            )

    # **Objective Function: Minimize Total Cost**
    cost = quicksum(
        (8 * hourly_wage) +                          # Guaranteed 8 hours pay
        (O[j] * overtime_rate) +                     # Overtime pay
        (FloorBonusFlag[j] * floor_bonus) +          # Floor bonus
        (SqFtExceedFlag[j] * hourly_wage * 8)        # Square footage penalty (wage doubles)
        for j in range(num_attendants)
    )

    model.setObjective(cost, GRB.MINIMIZE)

    # Solve the model
    model.optimize()

    # Collect results
    if model.status == GRB.OPTIMAL:
        optimal_cost = model.objVal
        total_overtime = sum(O[j].x for j in range(num_attendants))
        total_floors_assigned = sum(F_ik[j, k].x for j in range(num_attendants) for k in range(num_floors))
        total_floor_violations = sum(FloorBonusFlag[j].x for j in range(num_attendants))
        total_sq_ft_exceeded = sum(SqFtExceedFlag[j].x for j in range(num_attendants))

        return {
            "Overtime Rate Multiplier": overtime_rate_multiplier,
            "Optimal Cost": optimal_cost,
            "Total Overtime Hours": total_overtime,
            "Total Floors Assigned": total_floors_assigned,
            "Total Floor Violations": total_floor_violations,
            "Total Attendants Exceeding 3500 sq. ft.": total_sq_ft_exceeded,
        }
    else:
        return {"Error": "No optimal solution found"}

# **Run Both Scenarios**
scenario_1_5x = solve_scenario(1.5)  # 1.5× overtime pay
scenario_2x = solve_scenario(2.0)    # 2× overtime pay

# **Print Results for Comparison**
print("\n--- Scenario: 1.5× Overtime Pay ---")
for key, value in scenario_1_5x.items():
    print(f"{key}: {value}")

print("\n--- Scenario: 2× Overtime Pay ---")
for key, value in scenario_2x.items():
    print(f"{key}: {value}")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 220 rows, 560 columns and 2136 nonzeros
Model fingerprint: 0xaf4e9841
Variable types: 16 continuous, 544 integer (544 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve added 0 rows and 8 columns
Presolve removed 24 rows and 0 columns
Presolve time: 0.00s
Presolved: 196 rows, 568 columns, 1912 nonzeros
Variable types: 8 continuous, 560 integer (544 binary)
Found heuristic solution: objective 3441.4318182
Found heuristic solution: objective 3421.8977273

Root relaxation: objective 1.660511e+03, 151 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

e) Use Gurobi to solve the binary program. What is the optimal cost, and how many total overtime hours and floor violations (in excess of two) occur across all attendants?

In [5]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
hourly_wage = 25
overtime_rate = 1.5 * hourly_wage
max_overtime = 2  # Maximum allowed overtime hours
max_sq_ft = 3500  # Maximum square footage before pay doubles
floor_bonus = 75  # Additional payment per floor beyond 2

# Create the optimization model
model = Model("Hotel Cleaning Optimization")

# Decision Variables
X = model.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")
O = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
F = model.addVars(num_attendants, vtype=GRB.INTEGER, name="FloorsCleaned")
S = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")

# Binary variables to track floor bonus and square footage penalty
FloorBonusFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="FloorBonusFlag")
SqFtExceedFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="SqFtExceedFlag")

# Constraints

# Each room must be assigned to exactly one attendant
for i in range(num_rooms):
    model.addConstr(quicksum(X[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

# Track total cleaning time per attendant and enforce overtime constraints
for j in range(num_attendants):
    total_time = quicksum(X[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
    model.addConstr(total_time - O[j] <= 8, name=f"WorkHours_{j}")  # Total work time constraint

    # Overtime constraints
    model.addConstr(O[j] <= max_overtime, name=f"MaxOvertime_{j}")

# Ensure attendants clean between 2 and 4 floors per day
for j in range(num_attendants):
    unique_floors = quicksum(X[i, j] * hotels.loc[i, "Floor"] for i in range(num_rooms))
    model.addConstr(unique_floors >= 2, name=f"MinFloors_{j}")
    model.addConstr(unique_floors <= 4, name=f"MaxFloors_{j}")

    # Set FloorBonusFlag to 1 if more than 2 floors are cleaned
    model.addConstr(unique_floors - 2 <= FloorBonusFlag[j] * 100, name=f"FloorBonusConstraint_{j}")

# Ensure attendants do not exceed the square footage limit before pay doubles
for j in range(num_attendants):
    total_sq_ft = quicksum(X[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
    model.addConstr(S[j] == total_sq_ft, name=f"TotalSqFt_{j}")
    model.addConstr(S[j] <= max_sq_ft + SqFtExceedFlag[j] * 10000, name=f"MaxSqFt_{j}")

# Objective: Minimize cost including wages, overtime, and penalties
cost = quicksum(
    # Base pay: 8 hours guaranteed
    (8 * hourly_wage) +
    # Overtime pay
    (O[j] * overtime_rate) +
    # Floor bonus for attendants cleaning more than 2 floors
    (FloorBonusFlag[j] * floor_bonus) +
    # Pay doubling if square footage exceeds limit
    (SqFtExceedFlag[j] * hourly_wage * 8)
    for j in range(num_attendants)
)

model.setObjective(cost, GRB.MINIMIZE)

# Solve the model
model.optimize()

# Check the solver status before retrieving the objective value
if model.status == GRB.OPTIMAL:
    optimal_cost = model.objVal
    total_overtime = sum(O[j].x for j in range(num_attendants))
    total_floor_bonuses = sum(FloorBonusFlag[j].x for j in range(num_attendants))
    total_sq_ft_cleaned = sum(S[j].x for j in range(num_attendants))

    print(f"Optimal Cost: ${optimal_cost:.2f}")
    print(f"Total Overtime Hours: {total_overtime:.2f}")
    print(f"Total Floor Bonuses Applied: {total_floor_bonuses}")
    print(f"Total Square Footage Cleaned: {total_sq_ft_cleaned:.2f}")
    
elif model.status == GRB.INFEASIBLE:
    print("Model is infeasible! Try relaxing constraints or debugging infeasibilities.")
    model.computeIIS()  # Compute and print infeasibility details

elif model.status == GRB.UNBOUNDED:
    print("Model is unbounded! Check objective function and constraints.")

else:
    print(f"Optimization ended with status {model.status}. No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 456 columns and 2544 nonzeros
Model fingerprint: 0x014835c6
Variable types: 16 continuous, 440 integer (432 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 336 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Model is infeasible! Try relaxing constraints or debugging infeasibilities.
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 t

f) You will probably notice that Gurobi does not immediately provide the optimal solution when solving the model. Instead, it goes through numerous branch-and-bound iterations. While there are ways to enhance performance, one approach is to solve an approximation of the problem by relaxing the solution. Using Gurobi’s model.relax() procedure, what happens when you do this?

In [6]:
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
hourly_wage = 25
overtime_rate = 1.5 * hourly_wage
max_overtime = 2  # Maximum allowed overtime hours
max_sq_ft = 3500  # Maximum square footage before pay doubles
floor_bonus = 75  # Additional payment per floor beyond 2

# Create the optimization model
model = Model("Hotel Cleaning Optimization")

# Decision Variables
X = model.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")
O = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
F = model.addVars(num_attendants, vtype=GRB.INTEGER, name="FloorsCleaned")
S = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")

# Binary variables to track floor bonus and square footage penalty
FloorBonusFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="FloorBonusFlag")
SqFtExceedFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="SqFtExceedFlag")

# Constraints

# Each room must be assigned to exactly one attendant
for i in range(num_rooms):
    model.addConstr(quicksum(X[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

# Track total cleaning time per attendant and enforce overtime constraints
for j in range(num_attendants):
    total_time = quicksum(X[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
    model.addConstr(total_time - O[j] <= 8, name=f"WorkHours_{j}")  # Total work time constraint

    # Overtime constraints
    model.addConstr(O[j] <= max_overtime, name=f"MaxOvertime_{j}")

# Ensure attendants clean between 2 and 4 floors per day
for j in range(num_attendants):
    unique_floors = quicksum(X[i, j] * hotels.loc[i, "Floor"] for i in range(num_rooms))
    model.addConstr(unique_floors >= 2, name=f"MinFloors_{j}")
    model.addConstr(unique_floors <= 4, name=f"MaxFloors_{j}")

    # Set FloorBonusFlag to 1 if more than 2 floors are cleaned
    model.addConstr(unique_floors - 2 <= FloorBonusFlag[j] * 100, name=f"FloorBonusConstraint_{j}")

# Ensure attendants do not exceed the square footage limit before pay doubles
for j in range(num_attendants):
    total_sq_ft = quicksum(X[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
    model.addConstr(S[j] == total_sq_ft, name=f"TotalSqFt_{j}")
    model.addConstr(S[j] <= max_sq_ft + SqFtExceedFlag[j] * 10000, name=f"MaxSqFt_{j}")

# Objective: Minimize cost including wages, overtime, and penalties
cost = quicksum(
    # Base pay: 8 hours guaranteed
    (8 * hourly_wage) +
    # Overtime pay
    (O[j] * overtime_rate) +
    # Floor bonus for attendants cleaning more than 2 floors
    (FloorBonusFlag[j] * floor_bonus) +
    # Pay doubling if square footage exceeds limit
    (SqFtExceedFlag[j] * hourly_wage * 8)
    for j in range(num_attendants)
)

model.setObjective(cost, GRB.MINIMIZE)

# Solve the original model
print("Solving the original model...")
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"Optimal Cost: ${model.objVal:.2f}")
else:
    print("Original model did not reach an optimal solution.")

# **RELAXED MODEL SOLUTION**
print("\nSolving the relaxed model...")
relaxed_model = model.relax()
relaxed_model.optimize()

if relaxed_model.status == GRB.OPTIMAL:
    print(f"Relaxed Optimal Cost: ${relaxed_model.objVal:.2f}")
else:
    print("Relaxed model did not reach an optimal solution.")

Solving the original model...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 456 columns and 2544 nonzeros
Model fingerprint: 0x014835c6
Variable types: 16 continuous, 440 integer (432 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 336 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Original model did not reach an optimal solution.

Solving the relaxed model...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 l

g) Instead of using model.relax(), manually create a linear relaxation by converting the decision variables from binary to continuous with the appropriate bounds. What is the optimal cost? Compare this to the optimal solution of the binary program. What do your findings imply about using the solution of relaxed model as an approximation to the binary program?

In [7]:
from gurobipy import Model, GRB, quicksum

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
hourly_wage = 25
overtime_rate = 1.5 * hourly_wage
max_overtime = 2  # Maximum allowed overtime hours
max_sq_ft = 3500  # Maximum square footage before pay doubles
floor_bonus = 75  # Additional payment per floor beyond 2

# ======= ORIGINAL BINARY PROGRAM =======
model_binary = Model("Hotel Cleaning Optimization - Binary")

# Decision Variables (Binary)
X_bin = model_binary.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")
O_bin = model_binary.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
F_bin = model_binary.addVars(num_attendants, vtype=GRB.INTEGER, name="FloorsCleaned")
S_bin = model_binary.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")

# Binary variables to track floor bonuses and square footage penalties
FloorBonusFlag_bin = model_binary.addVars(num_attendants, vtype=GRB.BINARY, name="FloorBonusFlag")
SqFtExceedFlag_bin = model_binary.addVars(num_attendants, vtype=GRB.BINARY, name="SqFtExceedFlag")

# Constraints for Binary Model
for i in range(num_rooms):
    model_binary.addConstr(quicksum(X_bin[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

for j in range(num_attendants):
    total_time = quicksum(X_bin[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
    model_binary.addConstr(total_time - O_bin[j] <= 8, name=f"WorkHours_{j}")
    model_binary.addConstr(O_bin[j] <= max_overtime, name=f"MaxOvertime_{j}")

    unique_floors = quicksum(X_bin[i, j] * hotels.loc[i, "Floor"] for i in range(num_rooms))
    model_binary.addConstr(unique_floors >= 2, name=f"MinFloors_{j}")
    model_binary.addConstr(unique_floors <= 4, name=f"MaxFloors_{j}")
    model_binary.addConstr(unique_floors - 2 <= FloorBonusFlag_bin[j] * 100, name=f"FloorBonusConstraint_{j}")

    total_sq_ft = quicksum(X_bin[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
    model_binary.addConstr(S_bin[j] == total_sq_ft, name=f"TotalSqFt_{j}")
    model_binary.addConstr(S_bin[j] <= max_sq_ft + SqFtExceedFlag_bin[j] * 10000, name=f"MaxSqFt_{j}")

cost_binary = quicksum(
    (8 * hourly_wage) +
    (O_bin[j] * overtime_rate) +
    (FloorBonusFlag_bin[j] * floor_bonus) +
    (SqFtExceedFlag_bin[j] * hourly_wage * 8)
    for j in range(num_attendants)
)

model_binary.setObjective(cost_binary, GRB.MINIMIZE)

# Solve Binary Model
print("Solving the original binary program...")
model_binary.optimize()

# Store results
if model_binary.status == GRB.OPTIMAL:
    optimal_cost_binary = model_binary.objVal
    print(f"Binary Program Optimal Cost: ${optimal_cost_binary:.2f}")
else:
    print("Binary program did not find an optimal solution.")

# ======= MANUAL RELAXATION: LINEAR MODEL =======
model_relaxed = Model("Hotel Cleaning Optimization - Relaxed")

# Decision Variables (Relaxed Continuous)
X_rel = model_relaxed.addVars(num_rooms, num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="Assign")
O_rel = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
F_rel = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="FloorsCleaned")
S_rel = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")

# Relaxed "binary" variables (now continuous between 0 and 1)
FloorBonusFlag_rel = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="FloorBonusFlag")
SqFtExceedFlag_rel = model_relaxed.addVars(num_attendants, vtype=GRB.CONTINUOUS, lb=0, ub=1, name="SqFtExceedFlag")

# Constraints for Relaxed Model
for i in range(num_rooms):
    model_relaxed.addConstr(quicksum(X_rel[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

for j in range(num_attendants):
    total_time = quicksum(X_rel[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
    model_relaxed.addConstr(total_time - O_rel[j] <= 8, name=f"WorkHours_{j}")
    model_relaxed.addConstr(O_rel[j] <= max_overtime, name=f"MaxOvertime_{j}")

    unique_floors = quicksum(X_rel[i, j] * hotels.loc[i, "Floor"] for i in range(num_rooms))
    model_relaxed.addConstr(unique_floors >= 2, name=f"MinFloors_{j}")
    model_relaxed.addConstr(unique_floors <= 4, name=f"MaxFloors_{j}")
    model_relaxed.addConstr(unique_floors - 2 <= FloorBonusFlag_rel[j] * 100, name=f"FloorBonusConstraint_{j}")

    total_sq_ft = quicksum(X_rel[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
    model_relaxed.addConstr(S_rel[j] == total_sq_ft, name=f"TotalSqFt_{j}")
    model_relaxed.addConstr(S_rel[j] <= max_sq_ft + SqFtExceedFlag_rel[j] * 10000, name=f"MaxSqFt_{j}")

cost_relaxed = quicksum(
    (8 * hourly_wage) +
    (O_rel[j] * overtime_rate) +
    (FloorBonusFlag_rel[j] * floor_bonus) +
    (SqFtExceedFlag_rel[j] * hourly_wage * 8)
    for j in range(num_attendants)
)

model_relaxed.setObjective(cost_relaxed, GRB.MINIMIZE)

# Solve Relaxed Model
print("\nSolving the manually relaxed program...")
model_relaxed.optimize()

# Store results
if model_relaxed.status == GRB.OPTIMAL:
    optimal_cost_relaxed = model_relaxed.objVal
    print(f"Relaxed Program Optimal Cost: ${optimal_cost_relaxed:.2f}")
else:
    print("Relaxed program did not find an optimal solution.")

# **Comparison**
if model_binary.status == GRB.OPTIMAL and model_relaxed.status == GRB.OPTIMAL:
    print(f"\nComparison: Binary vs. Relaxed Solution")
    print(f"Binary Optimal Cost: ${optimal_cost_binary:.2f}")
    print(f"Relaxed Optimal Cost: ${optimal_cost_relaxed:.2f}")

Solving the original binary program...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 456 columns and 2544 nonzeros
Model fingerprint: 0x681028c8
Variable types: 16 continuous, 440 integer (432 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 336 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
Binary program did not find an optimal solution.

Solving the manually relaxed program...
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 

h) In the previous two questions, you explored two different methods for relaxing a MILP model. Explain why these approaches yield different results?

i) Now assume that attendants receive 2× their regular hourly wage (instead of time and a half) for the number of overtime hours worked. What is the optimal cost, and how many total overtime hours and floor violations (in excess of two) occur across all attendants? Compare this result to the optimal solution of the binary program in part (e). Was your intuition in part (d) correct?

In [8]:
from gurobipy import Model, GRB, quicksum

# Parameters
num_rooms = hotels.shape[0]
num_attendants = 8  # Given in the problem statement
hourly_wage = 25
overtime_rate = 2.0 * hourly_wage  # Now overtime is 2× the regular hourly wage
max_overtime = 2  # Maximum allowed overtime hours
max_sq_ft = 3500  # Maximum square footage before pay doubles
floor_bonus = 75  # Additional payment per floor beyond 2

# Create the optimization model
model = Model("Hotel Cleaning Optimization - Double Overtime Pay")

# Decision Variables
X = model.addVars(num_rooms, num_attendants, vtype=GRB.BINARY, name="Assign")
O = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="Overtime")
F = model.addVars(num_attendants, vtype=GRB.INTEGER, name="FloorsCleaned")
S = model.addVars(num_attendants, vtype=GRB.CONTINUOUS, name="SquareFootage")

# Binary variables to track floor bonus and square footage penalty
FloorBonusFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="FloorBonusFlag")
SqFtExceedFlag = model.addVars(num_attendants, vtype=GRB.BINARY, name="SqFtExceedFlag")

# Constraints

# Each room must be assigned to exactly one attendant
for i in range(num_rooms):
    model.addConstr(quicksum(X[i, j] for j in range(num_attendants)) == 1, name=f"AssignRoom_{i}")

# Track total cleaning time per attendant and enforce overtime constraints
for j in range(num_attendants):
    total_time = quicksum(X[i, j] * hotels.loc[i, "Cleaning_Time_Hours"] for i in range(num_rooms))
    model.addConstr(total_time - O[j] <= 8, name=f"WorkHours_{j}")  # Total work time constraint

    # Overtime constraints
    model.addConstr(O[j] <= max_overtime, name=f"MaxOvertime_{j}")

# Ensure attendants clean between 2 and 4 floors per day
for j in range(num_attendants):
    unique_floors = quicksum(X[i, j] * hotels.loc[i, "Floor"] for i in range(num_rooms))
    model.addConstr(unique_floors >= 2, name=f"MinFloors_{j}")
    model.addConstr(unique_floors <= 4, name=f"MaxFloors_{j}")

    # Set FloorBonusFlag to 1 if more than 2 floors are cleaned
    model.addConstr(unique_floors - 2 <= FloorBonusFlag[j] * 100, name=f"FloorBonusConstraint_{j}")

# Ensure attendants do not exceed the square footage limit before pay doubles
for j in range(num_attendants):
    total_sq_ft = quicksum(X[i, j] * hotels.loc[i, "Square_Feet"] for i in range(num_rooms))
    model.addConstr(S[j] == total_sq_ft, name=f"TotalSqFt_{j}")
    model.addConstr(S[j] <= max_sq_ft + SqFtExceedFlag[j] * 10000, name=f"MaxSqFt_{j}")

# Objective: Minimize cost including wages, overtime, and penalties
cost = quicksum(
    # Base pay: 8 hours guaranteed
    (8 * hourly_wage) +
    # Overtime pay (2× hourly wage for overtime hours)
    (O[j] * overtime_rate) +
    # Floor bonus for attendants cleaning more than 2 floors
    (FloorBonusFlag[j] * floor_bonus) +
    # Pay doubling if square footage exceeds limit
    (SqFtExceedFlag[j] * hourly_wage * 8)
    for j in range(num_attendants)
)

model.setObjective(cost, GRB.MINIMIZE)

# Solve the model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    optimal_cost = model.objVal
    total_overtime = sum(O[j].x for j in range(num_attendants))
    total_floor_violations = sum(FloorBonusFlag[j].x for j in range(num_attendants))

    print(f"Optimal Cost: ${optimal_cost:.2f}")
    print(f"Total Overtime Hours: {total_overtime:.2f}")
    print(f"Total Floor Violations (more than 2 floors): {total_floor_violations}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 456 columns and 2544 nonzeros
Model fingerprint: 0x882232d4
Variable types: 16 continuous, 440 integer (432 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+04]
  Objective range  [5e+01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 8 rows and 336 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
No optimal solution found.
